In [11]:
import csv
import matplotlib
import pandas as pd
import json
import os
import math
import re
from utils.postProcessing import *#img_ellipse_fitting
from utils.imageUtils import cropImage
from utils import imageUtils, postProcessing, DefectDetectionDataset, evaluation, visualization
import matplotlib.pyplot as plt

In [12]:
imagefolder = '/srv/home/whao/NextGen/new_debug/run/run1/datasets/dataprep/datasets/Loop_detection/images/'
root = '/srv/home/whao/NextGen/new_debug/run/run1/datasets/dataprep/datasets/Loop_detection'

# files = os.listdir('/srv/home/whao/NextGen/new_debug/run/run5/datasets/balloon/train/')    
dataset = DefectDetectionDataset(data_dir = root, split = 'test')

All_Image_Defects_List = list()
for item in range(len(dataset)):
    img, bboxes, label = dataset[item]
    subimages, bboxes = cropImage(img, bboxes)
    current_img_defect_List = list()
    for subim, bbox in zip(subimages, bboxes):
        defects_Dict = dict()
        defects_X_List = list()
        defects_Y_List = list()
        region1 = flood_fitting(subim)
        result = (int(region1['centroid'][0]+bbox[0]), int(region1['centroid'][1]+bbox[1]),
                    int(region1['minor_axis_length'] / 2), int(region1['major_axis_length'] / 2),
                    -region1['orientation'])
        rr,cc = draw.ellipse_perimeter(*result)
        # rr is row which is x in pixel
        # cc is column which is y in pixel
        # (rr,cc) represents one data points and there may be redundent data points
        assert len(rr) == len(cc)
#         flag = 0
        for i in range(len(cc)):
            if cc[i] <= 1024 and rr[i] <= 1024:
                defects_X_List.append(cc[i])
                defects_Y_List.append(rr[i])
#             if ((cc[i] > 1024 or rr[i] > 1024) and flag == 0):
#                 print(files[item])
#                 flag = 1
        # add defect's x,y corrdinates into the dictionary
        # update dictionary
        defects_Dict['X'] = defects_X_List
        defects_Dict['Y'] = defects_Y_List
        current_img_defect_List.append(defects_Dict)
    All_Image_Defects_List.append(current_img_defect_List)

In [13]:
len(All_Image_Defects_List)

30

In [14]:
def int64Toint32(list):
    for item in list:
        #print( type(item))
        item = int(item)
        #print( type(item) )
    return list

In [15]:
with open("via_region_data.json","w+") as jsonfile:
    with open("/srv/home/whao/NextGen/new_debug/run/run1/datasets/dataprep/datasets/Loop_detection/testimages.txt") as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    each_file = dict()
    for i in range(len(content)):
        img_i = All_Image_Defects_List[i]
        file_size = os.path.getsize(imagefolder + '/'+content[i])
        filename_size = content[i] + str(file_size)
        each_file[filename_size] = {'fileref':"",'size':file_size,'filename':content[i]}
        each_file[filename_size].update({'base64_img_data':"",'file_attributes':{},'regions':{}})
        each_bbox = each_file[filename_size]['regions']
        for j in range(len(img_i)):
            each_bbox.update({str(j):{'shape_attributes':{},'region_attributes':{}}})
            each_bbox[str(j)]['shape_attributes'] = {'name':"polygon"}
            each_bbox[str(j)]['shape_attributes'].update({'all_points_x':[],'all_points_y':[]})
            each_bbox[str(j)]['shape_attributes']['all_points_x'] = np.asarray(img_i[j]['X']).tolist()#int64Toint32(img_i[j]['X'])
            each_bbox[str(j)]['shape_attributes']['all_points_y'] = np.asarray(img_i[j]['Y']).tolist()#int64Toint32(img_i[j]['Y'])

    json.dump(each_file,jsonfile)